In [1]:
import numpy as np
from torch import Tensor
from sklearn.model_selection import KFold
from MLCode.utils import load_cup_data, np_cup_TR, plot_NN_TR_VAL
from sklearn.preprocessing import StandardScaler
from MLCode.NN import NN_HyperParameters
from MLCode.NN_cup import NN_Regressor, train_NN_cup, save_training


import matplotlib.pyplot as plt

df = load_cup_data()
X, Y = np_cup_TR(df)
Y_scaler = StandardScaler()
Y_scaler.fit(Y)
X, Y = Tensor(X), Tensor(Y)

# 10% test set
test_samples = X.shape[0] // 10

X_dev = X[:-test_samples]
Y_dev = Y[:-test_samples]

X_test = X[-test_samples:]
Y_test = Y[-test_samples:]

# 20% validation
val_samples = X_dev.shape[0] // 5
X_train = X_dev[:-val_samples]
Y_train = Y_dev[:-val_samples]

X_val = X_dev[-val_samples:]
Y_val = Y_dev[-val_samples:]

# train with K-fold cross-validation
def train_NN_K_validation(model, k_folds:int, X_dev, Y_dev, iters:int=None):
    """Train NN with k-fold cross-validation. If `iters` is set,
    instead of using all the folds, it will reach a number of `iters`
    training iterations.

    Returns `(MEE_mean, MEE_var, seconds, converged:Bool)`
    """
    kf = KFold(k_folds)
    val_errors = []
    seconds = 0
    converged = True
    for train_index, val_index in kf.split(X_dev):
        X_train, Y_train = X_dev[train_index], Y_dev[train_index]
        X_val, Y_val = X_dev[val_index], Y_dev[val_index]
        stats = train_NN_cup(net, X_train, Y_train, X_val, Y_val, 15,200,False)
        MEE, sec, conv = stats
        val_errors.append(MEE)
        seconds += sec
        converged = converged and conv
        if iters:
            iters -= 1
            if iters == 0: break

    val_errors = np.array(val_errors)
    MEE_mean = np.mean(val_errors)
    MEE_var = np.var(val_errors)
    return MEE_mean, MEE_var, seconds, converged


In [66]:
NN_HP = NN_HyperParameters(
    [10, 2000, 2000],
    lr=0.0001,
    beta1=0.9,
    beta2=0.999,
    weight_decay=0,
    mb_size=32,
)
net = NN_Regressor(2,NN_HP, Y_scaler)


# stats = train_NN_cup(net, X_train, Y_train, X_val, Y_val, 15,100,False)
stats = train_NN_K_validation(net, 5, X_dev, Y_dev, 3)
save_training(stats, NN_HP)

layers = [ 10, 2000, 2000,]
lr = 0.0001
beta1 = 0.9
beta2 = 0.999
weight_decay = 0
mb_size = 32

Val error achieved: 2.1349435251324977
 measured with variance: 0.460976141926033
Time (seconds): 378.5922475770001
Convergence: True


In [67]:
from pathlib import Path
from MLCode.conf import path_data
import pandas as pd

def get_model_val_err(directory: Path):
    """Returns validation error (mean and variance) from saved model directory.
    """
    info_file = directory / Path('infos.txt')
    info_text = info_file.read_text()
    lines = info_text.split('\n')
    mean_line = [x for x in lines if x.startswith('Val error') ][0]
    var_line = [x for x in lines if x.startswith(' measured') ][0]
    n_char_mean = len('Val error achieved: ')
    n_char_var = len('measured with variance: ')
    return float(mean_line[n_char_mean:]), float(var_line[n_char_var:])

def saved_NN_models():
    """Returns a pandas DataFrame with `model_name, MEE_mean, MEE_var`
    for each saved NN model."""

    NN_directory = path_data / Path('NN_training')
    models = []
    for dir in NN_directory.iterdir():
        val_err = get_model_val_err(dir)
        model = (dir.name, val_err[0], val_err[1])
        models.append(model)
    
    return pd.DataFrame(models, columns=['model_name', 'MEE_mean', 'MEE_var'])

df = saved_NN_models()
df.sort_values(by=['MEE_mean'])

,model_name,MEE_mean,MEE_var
24,500x500_1.0E-03,2.049151,0.452595
8,2000x2000_1.0E-03,2.073856,0.411696
18,2000x2000_1.0E-04,2.134944,0.460976
2,1000x1000_1.0E-03,2.182690,0.452786
15,200x200_1.0E-03,2.304236,0.268996
0,100x100_1.0E-03,2.560523,0.163153
20,200x200_1.0E-02,2.570066,0.054201
23,100x100_1.0E-02,2.589604,0.063337
27,3000_1.0E-03,2.630994,0.074615
1,50x50_1.0E-02,2.637516,0.048715
